In [19]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv
torch.manual_seed(0)
np.random.seed(0)

In [20]:
calculate_loss_over_all_values = False
input_window = 100
output_window = 5
batch_size = 10
device = torch.device("cuda")

Encoding

In [21]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]
       

Model

In [22]:
class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask


In [23]:
def create_input_sequences(input_data, tw):
    input_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = np.append(input_data[i:i+tw][:-output_window] , output_window * [0])
        train_label = input_data[i:i+tw]
        input_seq.append((train_seq ,train_label))
    return torch.FloatTensor(input_seq)

In [24]:
df = pd.read_csv("./dataset/merged.csv")
df = torch.Tensor(df.values)

In [25]:
df.shape

torch.Size([40178, 6])

In [26]:

def get_data():
    time        = np.arange(0, 400, 0.1)
    amplitude   = np.sin(time) + np.sin(time*0.05) +np.sin(time*0.12) *np.random.normal(-0.2, 0.2, len(time))
    
    series = read_csv('/home/arty/Desktop/NYU Spring 2022/6DOF/dataset/test.csv', header=0, index_col=0, parse_dates=False, squeeze=True)
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)

    
    sampels = 2800
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]
    train_sequence = create_input_sequences(train_data,input_window)
    train_sequence = train_sequence[:-output_window] 

    test_data = create_input_sequences(test_data,input_window)
    test_data = test_data[:-output_window] 

    return train_sequence.to(device),test_data.to(device)



In [28]:
def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) 
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target

In [29]:
def train(train_data):
    model.train()
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)        

        if calculate_loss_over_all_values:
            loss = criterion(output, targets)
        else:
            loss = criterion(output[-output_window:], targets[-output_window:])
    
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


In [30]:
def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            output = eval_model(data)    
            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()
            
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) 
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            # print(test_result.shape, test_data)
            # print(truth.shape, truth)
    len(test_result)

    # pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
    pyplot.plot(test_result-truth,color="green")
    pyplot.grid(True, which='both')
    # pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-epoch%d.png'%epoch)
    pyplot.close()
    
    return total_loss / i


In [31]:
def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    _ , data = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps,1):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    

    # pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    # pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-future%d.png'%steps)
    pyplot.close()

In [32]:
def evaluate(eval_model, data_source):
    eval_model.eval() 
    total_loss = 0.
    eval_batch_size = 200
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            if calculate_loss_over_all_values:
                total_loss += len(data[0])* criterion(output, targets).cpu().item()
            else:                                
                total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).cpu().item()            
    return total_loss / len(data_source)

train_data, val_data = get_data()
model = TransAm().to(device)

criterion = nn.MSELoss()
lr = 0.005 
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

best_val_loss = float("inf")
epochs = 100 

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    if(epoch % 10 is 0):
        val_loss = plot_and_loss(model, val_data,epoch)
        predict_future(model, val_data,200)
    else:
        val_loss = evaluate(model, val_data)
        
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    scheduler.step() 


<>:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_14839/560967404.py:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if(epoch % 10 is 0):
/home/arty/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |    53/  269 batches | lr 0.005000 |  4.27 ms | loss 6.67228 | ppl   790.19
| epoch   1 |   106/  269 batches | lr 0.005000 |  4.04 ms | loss 0.02530 | ppl     1.03
| epoch   1 |   159/  269 batches | lr 0.005000 |  4.34 ms | loss 0.00727 | ppl     1.01
| epoch   1 |   212/  269 batches | lr 0.005000 |  3.66 ms | loss 0.05185 | ppl     1.05
| epoch   1 |   265/  269 batches | lr 0.005000 |  3.67 ms | loss 0.04056 | ppl     1.04
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  1.63s | valid loss 0.35422 | valid ppl     1.43
-----------------------------------------------------------------------------------------
| epoch   2 |    53/  269 batches | lr 0.004802 |  3.78 ms | loss 0.04052 | ppl     1.04
| epoch   2 |   106/  269 batches | lr 0.004802 |  3.83 ms | loss 0.01932 | ppl     1.02
| epoch   2 |   159/  269 batches | lr 0.004802 |  4.69 ms | loss 0.01472 | ppl     1.01
| epoch   2 |   212/  269 batche

/home/arty/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch  11 |   106/  269 batches | lr 0.004004 |  3.64 ms | loss 0.06529 | ppl     1.07
| epoch  11 |   159/  269 batches | lr 0.004004 |  3.90 ms | loss 0.02500 | ppl     1.03
| epoch  11 |   212/  269 batches | lr 0.004004 |  3.87 ms | loss 0.03236 | ppl     1.03
| epoch  11 |   265/  269 batches | lr 0.004004 |  3.85 ms | loss 0.05052 | ppl     1.05
-----------------------------------------------------------------------------------------
| end of epoch  11 | time:  1.57s | valid loss 0.29212 | valid ppl     1.34
-----------------------------------------------------------------------------------------
| epoch  12 |    53/  269 batches | lr 0.003924 |  4.13 ms | loss 0.02111 | ppl     1.02
| epoch  12 |   106/  269 batches | lr 0.003924 |  4.54 ms | loss 0.08909 | ppl     1.09
| epoch  12 |   159/  269 batches | lr 0.003924 |  4.97 ms | loss 0.07513 | ppl     1.08
| epoch  12 |   212/  269 batches | lr 0.003924 |  5.09 ms | loss 0.03056 | ppl     1.03
| epoch  12 |   265/  269 batche

/home/arty/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

| epoch  71 |   106/  269 batches | lr 0.001191 |  3.80 ms | loss 0.15048 | ppl     1.16
| epoch  71 |   159/  269 batches | lr 0.001191 |  5.61 ms | loss 0.24290 | ppl     1.27
| epoch  71 |   212/  269 batches | lr 0.001191 |  3.93 ms | loss 0.06696 | ppl     1.07
| epoch  71 |   265/  269 batches | lr 0.001191 |  3.73 ms | loss 0.03818 | ppl     1.04
-----------------------------------------------------------------------------------------
| end of epoch  71 | time:  1.68s | valid loss 0.23073 | valid ppl     1.26
-----------------------------------------------------------------------------------------
| epoch  72 |    53/  269 batches | lr 0.001167 |  3.83 ms | loss 0.02886 | ppl     1.03
| epoch  72 |   106/  269 batches | lr 0.001167 |  4.24 ms | loss 0.14909 | ppl     1.16
| epoch  72 |   159/  269 batches | lr 0.001167 |  4.98 ms | loss 0.23895 | ppl     1.27
| epoch  72 |   212/  269 batches | lr 0.001167 |  4.48 ms | loss 0.06378 | ppl     1.07
| epoch  72 |   265/  269 batche

/home/arty/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch  81 |   106/  269 batches | lr 0.000973 |  3.74 ms | loss 0.14844 | ppl     1.16
| epoch  81 |   159/  269 batches | lr 0.000973 |  3.75 ms | loss 0.24506 | ppl     1.28
| epoch  81 |   212/  269 batches | lr 0.000973 |  3.74 ms | loss 0.06876 | ppl     1.07
| epoch  81 |   265/  269 batches | lr 0.000973 |  3.77 ms | loss 0.03741 | ppl     1.04
-----------------------------------------------------------------------------------------
| end of epoch  81 | time:  1.56s | valid loss 0.23070 | valid ppl     1.26
-----------------------------------------------------------------------------------------
| epoch  82 |    53/  269 batches | lr 0.000954 |  3.81 ms | loss 0.02920 | ppl     1.03
| epoch  82 |   106/  269 batches | lr 0.000954 |  3.65 ms | loss 0.14741 | ppl     1.16
| epoch  82 |   159/  269 batches | lr 0.000954 |  3.66 ms | loss 0.24309 | ppl     1.28
| epoch  82 |   212/  269 batches | lr 0.000954 |  4.00 ms | loss 0.06815 | ppl     1.07
| epoch  82 |   265/  269 batche

/home/arty/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch  91 |    53/  269 batches | lr 0.000795 |  3.82 ms | loss 0.02823 | ppl     1.03
| epoch  91 |   106/  269 batches | lr 0.000795 |  3.70 ms | loss 0.14729 | ppl     1.16
| epoch  91 |   159/  269 batches | lr 0.000795 |  5.00 ms | loss 0.24406 | ppl     1.28
| epoch  91 |   212/  269 batches | lr 0.000795 |  3.82 ms | loss 0.06930 | ppl     1.07
| epoch  91 |   265/  269 batches | lr 0.000795 |  3.84 ms | loss 0.03751 | ppl     1.04
-----------------------------------------------------------------------------------------
| end of epoch  91 | time:  1.64s | valid loss 0.22976 | valid ppl     1.26
-----------------------------------------------------------------------------------------
| epoch  92 |    53/  269 batches | lr 0.000779 |  3.89 ms | loss 0.02850 | ppl     1.03
| epoch  92 |   106/  269 batches | lr 0.000779 |  3.71 ms | loss 0.14745 | ppl     1.16
| epoch  92 |   159/  269 batches | lr 0.000779 |  3.78 ms | loss 0.24455 | ppl     1.28
| epoch  92 |   212/  269 batche